In [1]:
import json
from queue import Queue
import os
from bs4 import BeautifulSoup
import re
import networkx as nx
import operator
import copy

In [2]:
with open('../Wiki-Github-Link/Pagematched.json','r') as f:
    pagematched = json.load(f)
with open('../Wiki-Github-Link/page_github.json','r') as f:
    page_github = json.load(f)
with open('../Wiki-Github-Link/newcateinfo.json', 'r') as f:
    cateinfo = json.load(f)
    

In [6]:
pagestogithub = set([int(it) for it in pagematched.keys()])
pagegithublink = set([int(it) for it in page_github.keys()])


KeyError: 537

In [7]:
cateinfo = {int(k):v for k,v in cateinfo.items()}
usefulcates = set()
for cate, info in cateinfo.items():
    if 'pages' in info.keys():
        curpages = info['pages']
        if set(curpages).intersection(pagestogithub) or set(curpages).intersection(pagegithublink):
            usefulcates.add(cate)
usefulcateslist = list(usefulcates)
for cate in usefulcateslist:
    while cate != 0:
        cate = cateinfo[cate]['parent']
        if cate in usefulcates:
            break
        usefulcates.add(cate)
cateinfolimited = {}
usefulcateslist = sorted(list(usefulcates))
for cate in usefulcateslist:
    cateinfolimited[cate] = cateinfo[cate]
    children = cateinfo[cate]['children']
    cateinfolimited[cate]['children'] = [child for child in children if child in usefulcates]
with open('cateinfolimited.json', 'w') as opfile:
    json.dump(cateinfolimited, opfile, indent=1)
# for cate, info in cateinfolimited.items():
#     children = info['children']
#     cateinfolimited[cate]['children'] = [child for child in children if child in usefulcates]
# with open('cateinfolimited.json', 'w') as opfile:
#     json.dump(cateinfolimited, opfile, indent=1)

In [5]:
usefulcates

set()

In [3]:
with open('../Topic/Partition/freleaf.json', 'r') as f:
    freleaf = json.load(f)
freinfo = []


In [4]:
# 读取TopicCooccurrency-mt3.json
with open('../Topic/TopicCooccurrency-mt3.json', 'r') as ipfile:
    Coocjson = json.loads(ipfile.read())
G = nx.Graph()
for topic, towards in Coocjson.items():
    for toward, times in towards.items():
        if topic != toward:
            G.add_edge(topic, toward, weight=times)

In [12]:
key = 0
level = 0
parent_key = None
arrori = []
topickey = {}
for index1, p in enumerate(freleaf):
    if isinstance(p[0][0], list):
        parent_key = topickey[p[0][0][0]]
        level = arrori[parent_key]['level'] + 1
    elif p[0][0] in topickey:
        parent_key = topickey[p[0][0]]
        level = arrori[parent_key]['level'] + 1
    else:
        parent_key = None
    for index2, cur in enumerate(p):
        if isinstance(freleaf[index1][index2][0], list):
            community_members = cur[0]
            subG = G.subgraph(community_members)
            degrees = subG.degree(weight='weight')
            degree_dict = dict(degrees)
            sorted_degree = sorted(degree_dict.items(), key=operator.itemgetter(1), reverse=True)
            top_10_degree = sorted_degree[:10]
            # max_node = max(degrees, key=lambda x: x[1])
            item = {'key':key, 'parent':parent_key, 'labels':[it[0] for it in top_10_degree], 'level':level, 'childrens':[], 'topics':cur[0]}
            nkey = key
            key += 1
            arrori.append(item)
            for topic in [it[0] for it in freleaf[index1][index2][1:]]:
                titem = {'key':key, 'parent':nkey, 'labels':[topic], 'level':level + 1, 'childrens':[], 'topics':[topic]}
                key += 1
                arrori.append(item)
                item['childrens'].append(titem)
            if parent_key is not None:
                arrori[parent_key]['childrens'].append(item)
            continue
        subG = G.subgraph(cur)
        degrees = subG.degree(weight='weight')
        degree_dict = dict(degrees)
        sorted_degree = sorted(degree_dict.items(), key=operator.itemgetter(1), reverse=True)
        top_10_degree = sorted_degree[:10]
        max_node = max(degrees, key=lambda x: x[1])
        item = {'key':key, 'parent':parent_key, 'labels':[it[0] for it in top_10_degree], 'level':level, 'childrens':[], 'topics':cur}
        arrori.append(item)
        for topic in cur:
            topickey[topic] = key
        if parent_key is not None:
            arrori[parent_key]['childrens'].append(item)
        key += 1
arr = [it for it in arrori if it['level'] == 0]
with open('arr10.json', 'w') as opfile:
    opfile.write(json.dumps(arr, indent=1))

In [13]:
topicsubtree = {}
def dfs(node):
    for fre in node:
        if not fre['labels'][0] in topicsubtree:
            topicsubtree[fre['labels'][0]] = fre
        dfs(fre['childrens'])
dfs(arr)
with open('topicsubtree.json', 'w') as opfile:
    opfile.write(json.dumps(topicsubtree, indent=1))

In [24]:
with open('../Wiki-Github-Link/Pageleaf-redirect-encode.json', 'r') as f:
    pageleaf = json.load(f)

In [3]:
with open('topicsubtree.json', 'r') as f:
    topicsubtree = json.load(f)

In [26]:
nodes = []
edges = []
edgeid = 0
webnodes = Queue()
pageset = set()
def dfss(node, layer):
    if layer == 0:
        return
    global edgeid
    for child in node['childrens']:
        nodes.append({'id':'tp' + str(child['key']), 'label':child['labels'][0], 'group':2})
        edges.append({'id':str(edgeid), 'source':'tp' + str(node['key']), 'target':'tp' + str(child['key'])})
        edgeid += 1
        dfss(child, layer-1)
webnodes.put(795)
while not webnodes.empty():
    cate = webnodes.get()
    info = cateinfolimited[cate]
    children = info['children']
    nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    for child in children:
        edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
        edgeid += 1
        webnodes.put(child)
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            edgeid += 1
            pageset.add(page)
page = '967'
info = pagematched[page]
topic = info['topic']
if topic in topicsubtree:
    subtree = topicsubtree[topic]
    nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
    edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
    edgeid += 1
    dfss(subtree, -1)
    # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
    # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
    # edgeid += 1
else:
    nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
    edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
    edgeid += 1   

for page, info in pagematched.items():
    if not page in pageset:
        continue
    if page == '967':
        continue
    topic = info['topic']
    if topic in topicsubtree:
        subtree = topicsubtree[topic]
        nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
        edgeid += 1
        dfss(subtree, 2)
        # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
        # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
        # edgeid += 1
    else:
        nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
        edgeid += 1
graph = {'nodes':nodes, 'edges':edges}
with open('cpplibgraph2.json', 'w') as opfile:
    opfile.write(json.dumps(graph, indent=1))

In [ ]:
nodes = []
edges = []
edgeid = 0
def dfss(node, layer):
    if layer == 0:
        return
    global edgeid
    for child in node['childrens']:
        nodes.append({'id':'tp' + str(child['key']), 'label':child['labels'][0], 'group':2})
        edges.append({'id':str(edgeid), 'source':'tp' + str(node['key']), 'target':'tp' + str(child['key'])})
        edgeid += 1
        dfss(child, layer-1)
for cate, info in cateinfolimited.items():
    children = info['children']
    nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    for child in children:
        edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
        edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            edgeid += 1
page = 967
info = pagematched[page]
topic = info['topic']
if topic in topicsubtree:
    subtree = topicsubtree[topic]
    nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
    edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
    edgeid += 1
    dfss(subtree)
    # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
    # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
    # edgeid += 1
else:
    nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
    edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
    edgeid += 1
for page, info in pagematched.items():
    if page == 967:
        continue
    topic = info['topic']
    if topic in topicsubtree:
        subtree = topicsubtree[topic]
        nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
        edgeid += 1
        dfss(subtree, 2)
        # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
        # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
        # edgeid += 1
    else:
        nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
        edgeid += 1
graph = {'nodes':nodes, 'edges':edges}
with open('wholegraph.json', 'w') as opfile:
    opfile.write(json.dumps(graph, indent=1))

In [22]:
with open('cateinfoonlygithub.json', 'r') as f:
    cateinfolimited = json.load(f)

In [27]:
# cateinfoonlygithub = {}
for cate, info in cateinfolimited.items():
    cpages = []
    if 'pages' in info:
        cpages = info['pages']
        cpages = [page for page in cpages if str(page) in pagematched or str(page) in page_github]
    cateinfolimited[cate]['pages'] = cpages
with open('cateinfoonlygithub.json', 'w') as opfile:
    opfile.write(json.dumps(cateinfolimited, indent=1))

In [28]:
nodes = []
edges = []
edgeid = 0
def dfss(node, layer):
    if layer == 0:
        return
    global edgeid
    for child in node['childrens']:
        nodes.append({'id':'tp' + str(child['key']), 'label':child['labels'][0], 'group':2})
        edges.append({'id':str(edgeid), 'source':'tp' + str(node['key']), 'target':'tp' + str(child['key'])})
        edgeid += 1
        dfss(child, layer-1)
for cate, info in cateinfolimited.items():
    children = info['children']
    nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    for child in children:
        edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
        edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            edgeid += 1
for page, info in pagematched.items():
    topic = info['topic']
    if topic in topicsubtree:
        subtree = topicsubtree[topic]
        nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
        edgeid += 1
        dfss(subtree, 2)
        # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
        # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
        # edgeid += 1
    else:
        nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
        edgeid += 1
graph = {'nodes':nodes, 'edges':edges}
with open('wholegraphonlygithub.json', 'w') as opfile:
    opfile.write(json.dumps(graph, indent=1))

In [31]:
nodes = []
edges = []
edgeid = 0
webnodes = Queue()
pageset = set()
def dfss(node, layer):
    if layer == 0:
        return
    global edgeid
    for child in node['childrens']:
        nodes.append({'id':'tp' + str(child['key']), 'label':child['labels'][0], 'group':2})
        edges.append({'id':str(edgeid), 'source':'tp' + str(node['key']), 'target':'tp' + str(child['key'])})
        edgeid += 1
        dfss(child, layer-1)
webnodes.put(795)
while not webnodes.empty():
    cate = webnodes.get()
    info = cateinfolimited[cate]
    children = info['children']
    nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    for child in children:
        edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
        edgeid += 1
        webnodes.put(child)
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            edgeid += 1
            pageset.add(page)
# page = '967'
# info = pagematched[page]
# topic = info['topic']
# if topic in topicsubtree:
#     subtree = topicsubtree[topic]
#     nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
#     edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
#     edgeid += 1
#     dfss(subtree, -1)
#     # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
#     # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
#     # edgeid += 1
# else:
#     nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
#     edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
#     edgeid += 1   

for page, info in pagematched.items():
    if not int(page) in pageset:
        continue
    if page == '967':
        continue
    topic = info['topic']
    if topic in topicsubtree:
        subtree = topicsubtree[topic]
        nodes.append({'id':'tp' + str(subtree['key']), 'label':topic, 'group':2})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'tp' + str(subtree['key'])})
        edgeid += 1
        dfss(subtree, 2)
        # nodes.append({'id':'tp' + str(page), 'label':pageleaf[str(page)], 'group':1})
        # edges.append({'id':str(edgeid), 'source':str(topic), 'target':'pg' + str(page)})
        # edgeid += 1
    else:
        nodes.append({'id':'ts' + topic, 'label':topic, 'group':3})
        edges.append({'id':str(edgeid), 'source':'pg' + str(page), 'target':'ts' + topic})
        edgeid += 1
graph = {'nodes':nodes, 'edges':edges}
with open('cpplibgraph2o.json', 'w') as opfile:
    opfile.write(json.dumps(graph, indent=1))

In [9]:
topictreegraph = copy.deepcopy(topicsubtree)
def dfstopictree(subtree):
    subtree['id'] = 'tp' + str(subtree['key'])
    subtree['label'] = subtree['labels'][0]
    subtree['children'] = subtree['childrens']
    del subtree['childrens']
    del subtree['labels']
    del subtree['key']
    del subtree['parent']
    del subtree['level']
    del subtree['topics']
    for child in subtree['children']:
        dfstopictree(child)
for topic, subtree in topictreegraph.items():
    # subtree['id'] = 'tp' + str(subtree['key'])
    # subtree['label'] = subtree['labels'][0]
    # subtree['children'] = subtree['childrens']
    # del subtree['childrens']
    # del subtree['labels']
    # del subtree['key']
    # del subtree['parent']
    # del subtree['level']
    # del subtree['topics']
    dfstopictree(subtree)
with open('topictreegraph.json', 'w') as opfile:
    opfile.write(json.dumps(topictreegraph, indent=1))

In [11]:
def dfstopictag(subtree):
    subtree['group'] = 2
    for child in subtree['children']:
        dfstopictag(child)
for topic, subtree in topictreegraph.items():
    dfstopictag(subtree)
with open('topictreegraph.json', 'w') as opfile:
    opfile.write(json.dumps(topictreegraph, indent=1))

In [14]:
nodes = []
edges = []
edgeid = 0
wholegraph = {}
def dfss(node, layer):
    if layer == 0:
        return
    global edgeid
    for child in node['childrens']:
        nodes.append({'id':'tp' + str(child['key']), 'label':child['labels'][0], 'group':2})
        edges.append({'id':str(edgeid), 'source':'tp' + str(node['key']), 'target':'tp' + str(child['key'])})
        edgeid += 1
        dfss(child, layer-1)
for cate, info in reversed(cateinfolimited.items()):
    children = info['children']
    item = {'id':str(cate), 'label':info['name'], 'group':0, 'children':[wholegraph[child] for child in children if child in wholegraph]}
    wholegraph[int(cate)] = item
    # nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    # for child in children:
    #     edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
    #     edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            pageitem = {'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1, 'children':[]}
            if str(page) in pagematched:
                topic = pagematched[str(page)]['topic']
                if topic in topictreegraph:
                    pageitem['children'].append(topictreegraph[topic])
                else:
                    pageitem['children'].append({'id':'ts' + topic, 'label':topic, 'group':3, 'children':[]})
            item['children'].append(pageitem)
            # item['children'].append()
            # nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            # edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            # edgeid += 1
with open('wholegraphtree.json', 'w') as opfile:
    opfile.write(json.dumps(wholegraph, indent=1))

In [17]:
with open('wholegraphtree.json', 'w') as f:
    json.dump(wholegraph[0], f, indent=1)

In [38]:
with open('wholegraphtree.json', 'r') as f:
    wholegraph = json.load(f)

In [16]:
wholegraph.keys()

dict_keys([35285, 33275, 33268, 33258, 33187, 32875, 32872, 32685, 32681, 32413, 32233, 32231, 32228, 32204, 32172, 31818, 31803, 31637, 31607, 31502, 31482, 31343, 31342, 31334, 31241, 31237, 31071, 31008, 31007, 30986, 30983, 30963, 30880, 30814, 30808, 30804, 30799, 30798, 30642, 30364, 30359, 30248, 30242, 30200, 30191, 30184, 30183, 30181, 30180, 30178, 30176, 30175, 30171, 30165, 30162, 30047, 30046, 29688, 29319, 29285, 29257, 29249, 29247, 29240, 29236, 29214, 29093, 29092, 28934, 28900, 28893, 28892, 28844, 28841, 28791, 28477, 28467, 28435, 28179, 28178, 28177, 28174, 28168, 28167, 28162, 28161, 28154, 28153, 28151, 28150, 28149, 28148, 28147, 28118, 28109, 28107, 28106, 28098, 28079, 28050, 27919, 27720, 27676, 27675, 27674, 27647, 27639, 27610, 27607, 27599, 27598, 27591, 27587, 27573, 27435, 27431, 27420, 27419, 27415, 27414, 27406, 27403, 27391, 27373, 27367, 27359, 27354, 27329, 27328, 27309, 27184, 27150, 23209, 23167, 23143, 23100, 23097, 23091, 23080, 23078, 23066, 23

In [18]:
def dfsfind(tree, id):
    if tree['id'] == id:
        return tree
    for child in tree['children']:
        res = dfsfind(child, id)
        if res:
            return res
    return None
with open('jslibtree.json', 'w') as f:
    json.dump(dfsfind(wholegraph[0], '792'), f, indent=1)

In [39]:
def dfsextract(tree, id):
    for child in tree['children']:
        if child['id'] == id:
            subtree = copy.deepcopy(tree)
            subtree['children'] = [child]
            # for schild in subtree['children']:
            #     if schild['id'] != id:
            #         subtree['children'].remove(schild)
            return subtree
        res = dfsextract(child, id)
        if res:
            return res
    return None
vuetree = dfsextract(wholegraph, 'pg3437')
vuenode = vuetree['children'][0]
for child in vuenode['children']:
    for tchild in child['children']:
        tchild['children'] = []
with open('mvvmtree.json', 'w') as f:
    json.dump(vuetree, f, indent=1)

In [40]:
with open('../Topic/Prediction/repodict.json', 'r') as f:
    repodict = json.load(f)
topicrepo = {}
for repo, info in repodict.items():
    topics = info['topics']
    for topic in topics:
        if not topic in topicrepo:
            topicrepo[topic] = []
        topicrepo[topic].append({'name':repo, 'id':info['id'], 'stargazers_count':info['stargazers_count']})
with open('topicrepo.json', 'w') as f:
    json.dump(topicrepo, f, indent=1)

In [42]:
topicrepo = {k:[iv['id'] for iv in v] for k, v in topicrepo.items()}
with open('topicrepo.json', 'w') as f:
    json.dump(topicrepo, f, indent=1)

In [5]:
with open('topicrepo.json', 'r') as f:
    topicrepo = json.load(f)

In [4]:
topicrepo = {k:set(v) for k, v in topicrepo.items()}
with open('topicsubtree.json', 'r') as f:
    topictreegraphrepo = json.load(f)
with open('../Topic/Prediction/Repos-Info.json', 'r') as f:
    repoinfo = json.load(f)


In [8]:
def dfstopictree(subtree):
    subtree['id'] = 'tp' + str(subtree['key'])
    subtree['label'] = subtree['labels'][0]
    subtree['children'] = subtree['childrens']
    subtree['group'] = 2
    repos = set()
    for topic in subtree['topics']:
        repos = repos.union(topicrepo[topic])
    repos = list(sorted(repos, key=lambda x:repoinfo[str(x)]['stargazers_count'], reverse=True))
    repos = repos[:5]
    for repo in repos:
        subtree['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
    del subtree['childrens']
    del subtree['labels']
    del subtree['key']
    del subtree['parent']
    del subtree['level']
    del subtree['topics']
    for child in subtree['children']:
        dfstopictree(child)
for topic, subtree in topictreegraphrepo.items():
    dfstopictree(subtree)
with open('topictreegraphrepo.json', 'w') as opfile:
    opfile.write(json.dumps(topictreegraphrepo, indent=1))

KeyError: 'key'

In [ ]:
topicrepo = {k:set(v) for k, v in topicrepo.items()}
with open('topicsubtree.json', 'r') as f:
    topictreegraphrepo = json.load(f)


In [8]:
with open('../Topic/Prediction/Repos-Info.json', 'r') as f:
    repoinfo = json.load(f)


In [10]:
with open('topictreegraphrepo.json', 'r') as f:
    topictreegraphrepo = json.load(f)

In [11]:
def dfstopictree(subtree):
    if subtree['group'] == 4:
        return
    repos = subtree['repos']
    for repo in repos:
        subtree['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
    del subtree['repos']
    for child in subtree['children']:
        dfstopictree(child)
for topic, subtree in topictreegraphrepo.items():
    dfstopictree(subtree)
with open('topictreegraphrepo.json', 'w') as opfile:
    opfile.write(json.dumps(topictreegraphrepo, indent=1))

In [12]:
with open('wholegraphtree.json', 'r') as f:
    wholegraph = json.load(f)

In [4]:
with open('topictreegraph.json', 'r') as f:
    topictreegraph = json.load(f)

In [21]:

wholegraphrepo = {}

for cate, info in reversed(cateinfolimited.items()):
    children = info['children']
    item = {'id':str(cate), 'label':info['name'], 'group':0, 'children':[wholegraphrepo[child] for child in children if child in wholegraphrepo]}
    wholegraphrepo[int(cate)] = item
    # nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    # for child in children:
    #     edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
    #     edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            pageitem = {'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1, 'children':[]}
            if str(page) in pagematched:
                topic = pagematched[str(page)]['topic']
                if topic in topictreegraph:
                    pageitem['children'].append(topictreegraphrepo[topic])
                else:
                    ttitem = {'id':'ts' + topic, 'label':topic, 'group':3, 'children':[]}
                    if topic in topicrepo:
                        repos = topicrepo[topic]
                        repos = list(sorted(repos, key=lambda x:repoinfo[str(x)]['stargazers_count'], reverse=True))
                        repos = repos[:5]
                        for repo in repos:
                            ttitem['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
                    pageitem['children'].append(ttitem)
            item['children'].append(pageitem)
            # item['children'].append()
            # nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            # edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            # edgeid += 1
with open('wholegraphtreerepo.json', 'w') as opfile:
    opfile.write(json.dumps(wholegraphrepo[0], indent=1))

In [3]:
with open('wholegraphtreerepo.json', 'r') as f:
    wholegraphrepo = json.load(f)

In [33]:
def dfsextract(tree, id):
    if not 'children' in tree:
        return None
    for child in tree['children']:
        if child['id'] == id:
            subtree = copy.deepcopy(tree)
            subtree['children'] = [child]
            # for schild in subtree['children']:
            #     if schild['id'] != id:
            #         subtree['children'].remove(schild)
            return subtree
        res = dfsextract(child, id)
        if res:
            return res
    return None
vuetree = dfsextract(wholegraphrepo, 'pg1000')
vuenode = vuetree['children'][0]
for child in vuenode['children']:
    for tchild in child['children']:
        if tchild['group'] != 4:
            tchild['children'] = [ttchild for ttchild in tchild['children'] if ttchild['group'] == 4]
with open('vuerepo.json', 'w') as f:
    json.dump(vuetree, f, indent=1)

KeyError: 'children'

In [28]:
vuetree

{'id': '103',
 'label': 'Software design patterns',
 'group': 0,
 'children': [{'id': 'pg3437',
   'label': 'Model–view–viewmodel',
   'group': 1,
   'children': [{'id': 'tp721',
     'label': 'mvvm',
     'children': [{'id': 'tp2804',
       'label': 'mvvm',
       'children': [{'id': 'rp3412',
         'label': 'Blankj/AndroidUtilCode',
         'group': 4},
        {'id': 'rp43593', 'label': 'afollestad/material-dialogs', 'group': 4},
        {'id': 'rp66607', 'label': 'youth5201314/banner', 'group': 4},
        {'id': 'rp132857', 'label': 'BeyondDimension/SteamTools', 'group': 4},
        {'id': 'rp147063',
         'label': 'hehonghui/android-tech-frontier',
         'group': 4}],
       'group': 2},
      {'id': 'tp2805',
       'label': 'architecture',
       'children': [{'id': 'rp12916',
         'label': 'ryanmcdermott/clean-code-javascript',
         'group': 4},
        {'id': 'rp169529',
         'label': 'binhnguyennus/awesome-scalability',
         'group': 4},
        {

In [34]:
for child in vuenode['children']:
    for tchild in child['children']:
        if tchild['group'] != 4:
            tchild['children'] = [ttchild for ttchild in tchild['children'] if ttchild['group'] == 4]
with open('vuerepo.json', 'w') as f:
    json.dump(vuetree, f, indent=1)

In [31]:
labelset = set()
def dfswgt(tree):
    labelset.add(tree['label'])
    if 'children' in tree:
        for child in tree['children']:
            dfswgt(child)
dfswgt(wholegraph)
with open('labelset.json', 'w') as f:
    json.dump(list(labelset), f, indent=1)

In [36]:
topics = [
     "cors",
     "express",
     "mongoose",
     "nodemon",
     "passport",
     "passport.js",
     "jsonwebtoken",
     "joi",
     "express-boilerplate",
     "mean-stack",
     "mongoosejs"
    ]
reposet = set()
for topic in topics:
   if topic in topicrepo:
      reposet.update(topicrepo[topic])
key = 0
repolist = []
for repo in reposet:
   repolist.append({'id':key, 'reponame':repoinfo[repo]['full_name'], 'description':repoinfo[repo]['description']})
   key += 1
with open('corsrepolist.json', 'w') as f:
   json.dump(repolist, f, indent=1)

In [38]:
with open('corsrepolist.json', 'w') as f:
   json.dump(repolist, f, indent=1)

In [37]:
repolist = repolist[:60]

In [ ]:
for f in freleaf:
    for p in f:
        

In [5]:
topicreposet = {k:set(v) for k, v in topicrepo.items()}

In [4]:
with open('arr10.json', 'r') as f:
    arr10 = json.load(f)

In [6]:
arrrepo = []
def dfsfre(node):
    if node['childrens'] == []:
        newnode = copy.deepcopy(node)
        newnode['repos'] = set()
        for topic in node['topics']:
            newnode['repos'].update(topicreposet[topic])
        newnode['repos'] = list(newnode['repos'])
        return newnode
    if node['childrens'][0]['labels'][0] in set(node['topics']):
        crepos = set()
        irepos = set()
        newnode = copy.deepcopy(node)
        newnode['childrens'] = []
        for child in node['childrens']:
            nrepos = set()
            for topic in child['topics']:
                nrepos.update(topicreposet[topic])
            for repo in nrepos:
                if repo in crepos:
                    irepos.add(repo)
            crepos.update(nrepos)
            newnode['childrens'].append(dfsfre(child))
        newnode['repos'] = irepos
        newnode['repos'] = list(newnode['repos'])
        return newnode
    newnode = copy.deepcopy(node)
    newnode['repos'] = set()
    for topic in node['topics']:
        newnode['repos'].update(topicreposet[topic])
    newnode['repos'] = list(newnode['repos'])
    newnode['childrens'] = []
    for child in node['childrens']:
        newnode['childrens'].append(dfsfre(child))
    return newnode
for node in arr10:
    arrrepo.append(dfsfre(node))
with open('arrrepo.json', 'w') as f:
    json.dump(arrrepo, f, indent=1)

In [3]:
with open('arrrepo.json', 'r') as f:
    arrrepo = json.load(f)

In [ ]:

wholegraphrepo = {}

for cate, info in reversed(cateinfolimited.items()):
    children = info['children']
    item = {'id':str(cate), 'label':info['name'], 'group':0, 'children':[wholegraphrepo[child] for child in children if child in wholegraphrepo]}
    wholegraphrepo[int(cate)] = item
    # nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    # for child in children:
    #     edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
    #     edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            pageitem = {'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1, 'children':[]}
            if str(page) in pagematched:
                topic = pagematched[str(page)]['topic']
                if topic in topictreegraph:
                    pageitem['children'].append(topictreegraphrepo[topic])
                else:
                    ttitem = {'id':'ts' + topic, 'label':topic, 'group':3, 'children':[]}
                    if topic in topicrepo:
                        repos = topicrepo[topic]
                        repos = list(sorted(repos, key=lambda x:repoinfo[str(x)]['stargazers_count'], reverse=True))
                        repos = repos[:5]
                        for repo in repos:
                            ttitem['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
                    pageitem['children'].append(ttitem)
            item['children'].append(pageitem)
            # item['children'].append()
            # nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            # edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            # edgeid += 1
with open('wholegraphtreerepo.json', 'w') as opfile:
    opfile.write(json.dumps(wholegraphrepo[0], indent=1))

In [7]:
topicsubtree2 = {}
def dfs(node):
    for fre in node:
        if not fre['labels'][0] in topicsubtree2:
            topicsubtree2[fre['labels'][0]] = fre
        dfs(fre['childrens'])
dfs(arrrepo)
with open('topicsubtree2.json', 'w') as opfile:
    opfile.write(json.dumps(topicsubtree2, indent=1))

In [19]:
topictreegraph2 = copy.deepcopy(topicsubtree2)
def dfstopictree(subtree):
    if not 'key' in subtree:
        return
    subtree['id'] = 'tp' + str(subtree['key'])
    subtree['label'] = subtree['labels'][0]
    subtree['children'] = subtree['childrens']
    subtree['group'] = 2

    del subtree['childrens']
    del subtree['labels']
    del subtree['key']
    del subtree['parent']
    del subtree['level']
    del subtree['topics']
    for child in subtree['children']:
        dfstopictree(child)
    repos = subtree['repos']
    repos = list(sorted(repos, key=lambda x:repoinfo[str(x)]['stargazers_count'], reverse=True))
    if len(repos) > 15:
        repos = repos[:15]
    for repo in repos:
        subtree['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
    del subtree['repos']
for topic, subtree in topictreegraph2.items():
    dfstopictree(subtree)
with open('topictreegraph2.json', 'w') as opfile:
    opfile.write(json.dumps(topictreegraph2, indent=1))

In [15]:
subtree

{'id': 'tp30',
 'label': 'python',
 'children': [{'id': 'tp219',
   'label': 'python',
   'children': [{'id': 'tp1167',
     'label': 'python',
     'children': [{'id': 'tp3874',
       'label': 'python',
       'children': [{'id': 'tp8145',
         'label': 'python',
         'children': [{'id': 'tp8146',
           'label': 'utility-library',
           'children': [{'id': 'rp139735',
             'label': 'bharathgs/Awesome-pytorch-list',
             'group': 4},
            {'id': 'rp105972', 'label': 'vueuse/vueuse', 'group': 4},
            {'id': 'rp5169', 'label': 'andrewplummer/Sugar', 'group': 4},
            {'id': 'rp27348', 'label': 'oblac/jodd', 'group': 4},
            {'id': 'rp103388',
             'label': 'googlemaps/android-maps-utils',
             'group': 4},
            {'id': 'rp55053',
             'label': 'ml-tooling/best-of-python',
             'group': 4},
            {'id': 'rp175566',
             'label': 'magnusvk/counter_culture',
             'gro

In [14]:
topicsubtree2['deep-learning']

{'key': 0,
 'parent': None,
 'labels': ['deep-learning',
  'machine-learning',
  'python',
  'pytorch',
  'tensorflow',
  'nlp',
  'computer-vision',
  'data-science',
  'artificial-intelligence',
  'neural-network'],
 'level': 0,
 'childrens': [{'key': 30,
   'parent': 0,
   'labels': ['python',
    'python-3.x',
    'data-science',
    'visualization',
    'r',
    'django',
    'data-analysis',
    'jupyter',
    'pandas',
    'jupyter-notebook'],
   'level': 1,
   'childrens': [{'key': 219,
     'parent': 30,
     'labels': ['python',
      'python-3.x',
      'django',
      'python-asyncio',
      'flask',
      'fastapi',
      'pypi',
      'python-library',
      'python2',
      'django-rest-framework'],
     'level': 2,
     'childrens': [{'key': 1167,
       'parent': 219,
       'labels': ['python',
        'python-3.x',
        'pypi',
        'python-library',
        'python2',
        'pip',
        'python-2.x',
        'cython',
        'python-poetry',
        'cpyt

In [27]:

wholegraphrepo2 = {}

for cate, info in reversed(cateinfolimited.items()):
    children = info['children']
    item = {'id':str(cate), 'label':info['name'], 'group':0, 'children':[wholegraphrepo2[child] for child in children if child in wholegraphrepo2]}
    wholegraphrepo2[int(cate)] = item
    # nodes.append({'id':str(cate), 'label':info['name'], 'group':0})
    # for child in children:
    #     edges.append({'id':str(edgeid), 'source':str(cate), 'target':str(child)})
    #     edgeid += 1
    if 'pages' in info:
        pages = info['pages']
        for page in pages:
            if not str(page) in pageleaf:
                continue
            pageitem = {'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1, 'children':[]}
            if str(page) in pagematched:
                topic = pagematched[str(page)]['topic']
                if topic in topictreegraph2:
                    pageitem['children'].append(topictreegraph2[topic])
                else:
                    ttitem = {'id':'ts' + topic, 'label':topic, 'group':3, 'children':[]}
                    if topic in topicrepo:
                        repos = topicrepo[topic]
                        repos = list(sorted(repos, key=lambda x:repoinfo[str(x)]['stargazers_count'], reverse=True))
                        repos = repos[:15]
                        for repo in repos:
                            ttitem['children'].append({'id':'rp' + repo, 'label':repoinfo[str(repo)]['full_name'], 'group':4})
                    pageitem['children'].append(ttitem)
            item['children'].append(pageitem)
            # item['children'].append()
            # nodes.append({'id':'pg' + str(page), 'label':pageleaf[str(page)], 'group':1})
            # edges.append({'id':str(edgeid), 'source':str(cate), 'target':'pg' + str(page)})
            # edgeid += 1
with open('wholegraphtreerepo2.json', 'w') as opfile:
    opfile.write(json.dumps(wholegraphrepo2[0], indent=1))

In [1]:
l = [0, 1, 2, 3, 4]
print(l[:10])

[0, 1, 2, 3, 4]


In [ ]:
file = r'../Wiki-Pageleaves'

for root, dirs, files in os.walk(file):
    if root != file:
        break
    for file in files:
        path = os.path.join(root, file)
        with open(path, 'r', encoding='utf-8') as f:
            html = f.read()
        

In [9]:
def dfs(tree):
    topics = set()
    topics.add(tree['label'])
    for child in tree['children']:
        topics |= dfs(child)
    return topics

notoccurred = copy.deepcopy(topictreegraph)
for page, info in pagematched.items():
    topic = info['topic']
    if topic in notoccurred:
        del notoccurred[topic]
occurredtopic = set()
while True:
    for tree, info in notoccurred.items():
        for child in info['children']:
            occurredtopic |= dfs(child)
    l = len(notoccurred)
    notoccurred = {k:v for k, v in notoccurred.items() if not k in occurredtopic}
    if len(notoccurred) == l:
        break


TypeError: dump() missing 1 required positional argument: 'fp'

In [11]:
with open('notoccurred.json', 'w') as f:
    json.dump(notoccurred, f, indent=1)


In [ ]:
for topic, info in topictreegraph.items():
    if topic in notoccurred:
        continue
    topics = set(dfs(info))
    notoccurred = {k:v for k, v in notoccurred.items() if not k in topics}

In [ ]:
repo = 'NoneDay/CellReport'
with open('../Topic/Prediction/repodict.json', 'r') as f:
    repodict = json.load(f)
topics = repodict[repo]['topics']
def dfsarr(tree):
    
repoid = '167720'
relatedtopics = set()
for topic, repos in topicrepo.items():
    if repoid in set(repos):
        relatedtopics.add()

In [3]:
with open('../Topic/Partition/fre.json', 'r') as f:
    fre = json.load(f)

In [ ]:
key = 0
level = 0
parent_key = None
arrori = []
topickey = {}
for index1, p in enumerate(fre):
    if p[0][0] in topickey:
        parent_key = topickey[p[0][0]]
        level = arrori[parent_key]['level'] + 1
    else:
        parent_key = None
    for index2, cur in enumerate(p):
        subG = G.subgraph(cur)
        degrees = subG.degree(weight='weight')
        degree_dict = dict(degrees)
        sorted_degree = sorted(degree_dict.items(), key=operator.itemgetter(1), reverse=True)
        top_10_degree = sorted_degree[:10]
        max_node = max(degrees, key=lambda x: x[1])
        item = {'key':key, 'parent':parent_key, 'labels':[it[0] for it in top_10_degree], 'level':level, 'childrens':[], 'topics':cur}
        arrori.append(item)
        for topic in cur:
            topickey[topic] = key
        if parent_key is not None:
            arrori[parent_key]['childrens'].append(item)
        key += 1
arr = [it for it in arrori if it['level'] == 0]
with open('arr10fre.json', 'w') as opfile:
    opfile.write(json.dumps(arr, indent=1))

In [6]:
clurepos = {}
def dfsarr10(tree):
    if tree['childrens'] == []:
        topics = tree['topics']
        reposet = set()
        for topic in topics:
            reposet.update(topicrepo[topic])
        clurepos[tree['key']] = list(reposet)
    else:
        for child in tree['childrens']:
            dfsarr10(child)
for subtree in arr10:
    dfsarr10(subtree)
with open('clurepos.json', 'w') as opfile:
    opfile.write(json.dumps(clurepos, indent=1))

In [7]:
repoclus = {}
for clu, repos in clurepos.items():
    for repo in repos:
        if not repo in repoclus:
            repoclus[repo] = []
        repoclus[repo].append(clu)
with open('repoclus.json', 'w') as opfile:
    opfile.write(json.dumps(repoclus, indent=1))

In [10]:
cluleafinfo = {}
def dfswholetree(tree):
    if not 'children' in tree:
        return
    if tree['children'] != [] and tree['children'][0]['id'].startswith('rp'):
        if not tree['id'] in cluleafinfo:
            cluleafinfo[tree['id']] = {'label':tree['label'], 'group':2}
    else:
        for child in tree['children']:
            dfswholetree(child)
dfswholetree(wholegraphrepo)
with open('cluleafinfo.json', 'w') as opfile:
    opfile.write(json.dumps(cluleafinfo, indent=1))